In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
import json

In [4]:
with open("list_of_domains.txt") as f:
    domains = f.read().splitlines()

In [5]:
domains

['Data',
 'Software Development and Programming',
 'Graphic Design and Multimedia',
 'Computer Systems and Networks',
 'Telecommunications',
 'Electrical Engineering',
 'Mechanical Engineering',
 'Chemistry and Chemical Engineering',
 'Agriculture and Food',
 'Aerospace',
 'Logistics and Transportation',
 'Production and Manufacturing',
 'Commerce',
 'Marketing',
 'Finance and Banking',
 'Accounting',
 'Human Resources',
 'Consulting',
 'Quality Assurance and Control',
 'Administrative Support and Reception',
 'Hospitality and Tourism',
 'Education and Training',
 'Healthcare and Medicine',
 'Legal and Law']

In [16]:
url_search_base = "https://www.hellowork.com/fr-fr/emploi/recherche.html?k="
urls = {}

In [17]:
for domain in domains:
    urls[domain] = url_search_base + domain.replace(" ", "+")

In [18]:
urls

{'Data': 'https://www.hellowork.com/fr-fr/emploi/recherche.html?k=Data',
 'Software Development and Programming': 'https://www.hellowork.com/fr-fr/emploi/recherche.html?k=Software+Development+and+Programming',
 'Graphic Design and Multimedia': 'https://www.hellowork.com/fr-fr/emploi/recherche.html?k=Graphic+Design+and+Multimedia',
 'Computer Systems and Networks': 'https://www.hellowork.com/fr-fr/emploi/recherche.html?k=Computer+Systems+and+Networks',
 'Telecommunications': 'https://www.hellowork.com/fr-fr/emploi/recherche.html?k=Telecommunications',
 'Electrical Engineering': 'https://www.hellowork.com/fr-fr/emploi/recherche.html?k=Electrical+Engineering',
 'Mechanical Engineering': 'https://www.hellowork.com/fr-fr/emploi/recherche.html?k=Mechanical+Engineering',
 'Chemistry and Chemical Engineering': 'https://www.hellowork.com/fr-fr/emploi/recherche.html?k=Chemistry+and+Chemical+Engineering',
 'Agriculture and Food': 'https://www.hellowork.com/fr-fr/emploi/recherche.html?k=Agricultur

In [37]:
# urls = {}
urls["Graphic Design and Multimedia"] = "https://www.hellowork.com/fr-fr/emploi/recherche.html?k=Graphic+Designer&k_autocomplete=&l=&l_autocomplete=&ray=20&msa=&cod=all&d=all&c_idesegal="
urls["Agriculture and Food"] = "https://www.hellowork.com/fr-fr/emploi/recherche.html?k=Agriculture&k_autocomplete=&l=&l_autocomplete=&ray=20&msa=&cod=all&d=all&c_idesegal="
urls["Logistics and Transportation"] = "https://www.hellowork.com/fr-fr/emploi/recherche.html?k=Logistics&k_autocomplete=&l=&l_autocomplete=&ray=20&msa=&cod=all&d=all&c_idesegal="
urls["Chemistry and Chemical Engineering"] = "https://www.hellowork.com/fr-fr/emploi/recherche.html?k=Chemical&k_autocomplete=&l=&l_autocomplete=&ray=20&msa=&cod=all&d=all&c_idesegal="
urls["Finance and Banking"] = "https://www.hellowork.com/fr-fr/emploi/recherche.html?k=Finance&k_autocomplete=&l=&l_autocomplete=&ray=20&msa=&cod=all&d=all&c_idesegal="
urls["Education and Training"] = "https://www.hellowork.com/fr-fr/emploi/recherche.html?k=Education&k_autocomplete=&l=&l_autocomplete=&ray=20&msa=&cod=all&d=all&c_idesegal="
urls["Legal and Law"] = "https://www.hellowork.com/fr-fr/emploi/recherche.html?k=Legal&k_autocomplete=&l=&l_autocomplete=&ray=20&msa=&cod=all&d=all&c_idesegal="
urls["Healthcare and Medicine"] = "https://www.hellowork.com/fr-fr/emploi/recherche.html?k=M%C3%A9decin&k_autocomplete=http%3A%2F%2Fwww.rj.com%2FCommun%2FPost%2FMedecin&l=&l_autocomplete=&ray=20&msa=&cod=all&d=all&c_idesegal="
urls["Hospitality and Tourism"] = "https://www.hellowork.com/fr-fr/emploi/recherche.html?k=Hospitality&k_autocomplete=&l=&l_autocomplete=&ray=20&msa=&cod=all&d=all&c_idesegal="

In [19]:
cols = ["description", "domain"]

In [20]:
# offres_pd = pd.read_csv("hellowork_raw.csv")
# offres_pd = pd.DataFrame(columns=cols)

In [35]:
begin = 1
max_pages_per_cat = 11

In [39]:
for cat, url in urls.items():
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "lxml")

    for p in range(begin, max_pages_per_cat+1):
        url_page = url + "&p=" + str(p)
        hdr = {'User-Agent': 'Mozilla/5.0'}
        page = requests.get(url_page, headers=hdr)
        soup = BeautifulSoup(page.text, "lxml")
        results = soup.find("ul", class_="crushed")
        job_elements = results.find_all("li", class_="!tw-mb-6")
        for i, job_element in enumerate(job_elements):
            job_link = "https://www.hellowork.com/fr-fr/emplois/" + job_element.find("div")["id"] + ".html"
            offre_page = requests.get(job_link, headers=hdr)

            offre_soup = BeautifulSoup(offre_page.text, "lxml", parse_only=SoupStrainer("script", type="application/ld+json"))
            json_data = offre_soup.find_all("script", type="application/ld+json")
            if json_data:
                try:
                    json_data = json.loads(json_data[3].text, strict=False)
                    titre = json_data["title"]
                    description = json_data["description"]
                    description = titre + " " + description
                    
                    offres_pd = pd.concat([offres_pd, pd.DataFrame([[description, cat]], columns=cols)])
                    offres_pd.to_csv("hellowork_raw.csv", index=False)
                    print(f"Page: {p} | Offre: {(p - 1) * 21 + i + 1}", titre, cat)
                except Exception as e:
                    print(f"Page: {p} | Offre: {(p - 1) * 21 + i + 1}", e, e.__traceback__.tb_lineno)
                    continue
            else:
                print(f"Page: {p} | Offre: {(p - 1) * 20 + i + 1}", "Not found")

Page: 1 | Offre: 1 Graphic Designer en Alternance H/F Graphic Design and Multimedia
Page: 1 | Offre: 2 Graphic Designer en Alternance H/F Graphic Design and Multimedia
Page: 1 | Offre: 3 Graphic Designer en Alternance H/F Graphic Design and Multimedia
Page: 1 | Offre: 4 Alternant - Assistant Graphic & Motion Designer H/F Graphic Design and Multimedia
Page: 1 | Offre: 5 Designer Graphic en Alternance H/F Graphic Design and Multimedia
Page: 1 | Offre: 6 Graphiste H/F Graphic Design and Multimedia
Page: 1 | Offre: 7 Infographiste H/F Graphic Design and Multimedia
Page: 1 | Offre: 8 Graphiste H/F Graphic Design and Multimedia
Page: 1 | Offre: 9 Infographiste H/F Graphic Design and Multimedia
Page: 1 | Offre: 10 Graphiste H/F Graphic Design and Multimedia
Page: 1 | Offre: 11 Infographiste H/F Graphic Design and Multimedia
Page: 1 | Offre: 12 Infographiste H/F Graphic Design and Multimedia
Page: 1 | Offre: 13 Graphiste H/F Graphic Design and Multimedia
Page: 1 | Offre: 14 Infographiste H/F G

ConnectionError: HTTPSConnectionPool(host='www.hellowork.com', port=443): Max retries exceeded with url: /fr-fr/emplois/40025954.html (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000235B62D1950>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))